In [2]:
# ============================
# Financial Data Dashboard
# ============================

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Make folders if not present
os.makedirs("data", exist_ok=True)
os.makedirs("plots", exist_ok=True)

# ----------------------------
# 1. Parameters
# ----------------------------
tickers = ["BTC-USD", "SPY", "GLD", "TLT"]
start = "2022-01-01"
end = None   # fetch until today

# ----------------------------
# 2. Fetch and clean prices
# ----------------------------
raw = yf.download(tickers, start=start, end=end, auto_adjust=True)

# yfinance gives MultiIndex cols, keep only Close
if isinstance(raw.columns, pd.MultiIndex):
    prices = raw["Close"].dropna(how="all")
else:
    prices = raw.dropna(how="all")

prices.to_csv("data/prices.csv")

# ----------------------------
# 3. Compute returns
# ----------------------------
returns = prices.pct_change().dropna()
returns.to_csv("data/returns.csv")

# ----------------------------
# 4. Normalized performance plot
# ----------------------------
normed = prices / prices.iloc[0] * 100

plt.figure(figsize=(12,6))
for col in normed.columns:
    sns.lineplot(x=normed.index, y=normed[col], label=col, linewidth=2)

sns.set_style("dark")
plt.title("Normalized Performance (Start = 100)", fontsize=14, weight="bold")
plt.ylabel("Index (100 = start)")
plt.xlabel("Date")
plt.legend(frameon=False)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("plots/normalized_performance.png", dpi=300)
plt.close()

# ----------------------------
# 5. Correlation heatmap
# ----------------------------
corr = returns.corr()

plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="vlag", vmin=-1, vmax=1)
plt.title("Correlation Matrix of Daily Returns")
plt.tight_layout()
plt.savefig("plots/correlation_heatmap.png", dpi=300)
plt.close()

# ----------------------------
# 6. Rolling correlation BTC vs SPY
# ----------------------------
rolling_corr = returns["BTC-USD"].rolling(30).corr(returns["SPY"])

sns.set_style("dark")
plt.figure(figsize=(12,4))
sns.lineplot(x=rolling_corr.index, y=rolling_corr, linewidth=2, color="blue")
plt.axhline(0, ls="--", color="grey", alpha=0.7)
plt.title("30-Day Rolling Correlation: BTC vs SPY")
plt.ylabel("Correlation")
plt.tight_layout()
plt.savefig("plots/rolling_corr_btc_spy.png", dpi=300)
plt.close()

# ----------------------------
# 7. Drawdowns
# ----------------------------
def drawdown(series):
    cummax = series.cummax()
    return (series - cummax) / cummax

sns.set_style("dark")
plt.figure(figsize=(12,4))
for col in prices.columns:
    dd = drawdown(prices[col])
    sns.lineplot(x=dd.index, y=dd, label=col)

plt.title("Drawdowns from Peak")
plt.ylabel("Drawdown")
plt.xlabel("Date")
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("plots/drawdowns.png", dpi=300)
plt.close()

print("Data and plots saved in /data and /plots")

# ----------------------------
# 8. Summary Table of Returns & Volatility
# ----------------------------

summary = pd.DataFrame({
    "Cumulative Return": (1 + returns).prod() - 1,
    "Annualized Volatility": returns.std() * np.sqrt(252)
})

summary.to_csv("data/summary_table.csv")

# Save as PNG table
fig, ax = plt.subplots(figsize=(6,2))
ax.axis("off")
tbl = ax.table(cellText=summary.round(4).reset_index().values,
               colLabels=["Asset", "Cumulative Return", "Ann. Volatility"],
               loc="center")
tbl.auto_set_font_size(False)
tbl.set_fontsize(9)
tbl.scale(1.2, 1.2)
plt.tight_layout()
plt.savefig("plots/summary_table.png", dpi=300, bbox_inches="tight")
plt.close()

print("Summary table saved to data/summary_table.csv and plots/summary_table.png")


[*********************100%***********************]  4 of 4 completed
/var/folders/pc/pnwn7khs08xc5_mdksljl66m0000gn/T/ipykernel_13202/3039471583.py:39: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


Data and plots saved in /data and /plots
Summary table saved to data/summary_table.csv and plots/summary_table.png
